In [41]:
!pip install -q  transformers==4.30.1 datasets evaluate thaixtransformers
!pip install peft==0.10.0

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [159 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.9-universal2-cpython-312/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.macosx-10.9-universal2-cpython-312/tokenizers
      creating build/lib.macosx-10.9-universal2-cpython-312/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.macosx-10.9-universal2-cpython-312/tokenizers/models
      creating build/lib.macosx-10.9-universal2-cpython-312/tokenizers/decoders
      copying py_src/tokenizers/decoders/__init__.py -> build/lib.macosx-10.9-universal2-cpython-312/tokenizers/decoders
      creating build/lib.macosx-10.9-universal2-cpython-312/tokenizers/normalizers
      copying py_src/tokenizers/normalizers/__init__.py -> build/lib.macosx-10.9-universal2-cpython-312/tokenizers/

In [1]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
from huggingface_hub import notebook_login

notebook_login()

/Users/idhibhatpankam/Code/courses/DATA-SCI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,comment,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,0,0,0,0,0,0,0,0,0,0,0,0
1,1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,0,0,0,0,0,0,0,0,0,0,0,0
2,2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,0,0,0,0,0,0,0,0,0,0,0,0
3,3,ปัญหาน้ำท่วมในซอยสวนพลู 1,0,0,0,0,0,0,0,0,0,0,0,0
4,4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
train_df.describe()

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
count,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000
mean,102310.500000,0.209635,0.019969,0.000049,0.016034,0.000117,0.167944,0.034542,0.000406,0.024176,0.018478,0.000059,0.023839
std,59069.427728,0.407049,0.139892,0.006991,0.125608,0.010829,0.373817,0.182616,0.020136,0.153597,0.134672,0.007658,0.152548
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51155.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,102310.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,153465.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,204621.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
def drop_rows(df):
    # drop rows with missing values, duplicates
    print(f"Before dropna: {df.shape}")
    df.dropna(inplace=True)
    df.drop_duplicates("comment", keep="first", inplace=True)
    df = df.reset_index(drop=True)
    print(f"After dropna: {df.shape}")

drop_rows(train_df)
drop_rows(test_df)
print(train_df.shape, test_df.shape)

Before dropna: (204622, 14)
After dropna: (204622, 14)
Before dropna: (10810, 2)
After dropna: (10810, 2)
(204622, 14) (10810, 2)


In [ ]:
contacts = train_df.columns[2:]
train_df['label'] = train_df[contacts].apply(lambda x: list(x), axis=1)

In [ ]:
train_df.drop(contacts, axis=1, inplace=True)
train_df.drop("id", axis=1, inplace=True)
test_df.drop("id", axis=1, inplace=True)
train_df.head()

,comment,label
0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,ปัญหาน้ำท่วมในซอยสวนพลู 1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [30]:
train_df['label'].value_counts()

label
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    102333
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]     42212
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]     32543
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]      5803
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]      4598
                                         ...  
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]         1
[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]         1
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]         1
[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]         1
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]         1
Name: count, Length: 73, dtype: int64

In [31]:
contacts

Index(['สำนักงานตำรวจแห่งชาติ', 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
       'สภาเด็กและเยาวชนกรุงเทพมหานคร', 'กรมควบคุมมลพิษ', 'กรมสรรพสามิต',
       'การไฟฟ้านครหลวง', 'กรมทางหลวง', 'สำนักงานประกันสุขภาพแห่งชาติ',
       'การประปานครหลวง', 'คณะกรรมการการพัฒนาเศรษฐกิจ',
       'กระทรวงการท่องเที่ยวและกีฬา', 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'],
      dtype='object')

In [36]:
train_data = train_df.copy().to_numpy()

label_2_num_map = dict(zip(contacts, range(len(contacts))))
num_2_label_map = dict(zip(range(len(contacts)), contacts))

print("Create Mappings")
display(num_2_label_map)
display(label_2_num_map)

# print("Before Mappings")
# display(data[:, 1])
# data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'สำนักงานตำรวจแห่งชาติ',
 1: 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
 2: 'สภาเด็กและเยาวชนกรุงเทพมหานคร',
 3: 'กรมควบคุมมลพิษ',
 4: 'กรมสรรพสามิต',
 5: 'การไฟฟ้านครหลวง',
 6: 'กรมทางหลวง',
 7: 'สำนักงานประกันสุขภาพแห่งชาติ',
 8: 'การประปานครหลวง',
 9: 'คณะกรรมการการพัฒนาเศรษฐกิจ',
 10: 'กระทรวงการท่องเที่ยวและกีฬา',
 11: 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'}

{'สำนักงานตำรวจแห่งชาติ': 0,
 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย': 1,
 'สภาเด็กและเยาวชนกรุงเทพมหานคร': 2,
 'กรมควบคุมมลพิษ': 3,
 'กรมสรรพสามิต': 4,
 'การไฟฟ้านครหลวง': 5,
 'กรมทางหลวง': 6,
 'สำนักงานประกันสุขภาพแห่งชาติ': 7,
 'การประปานครหลวง': 8,
 'คณะกรรมการการพัฒนาเศรษฐกิจ': 9,
 'กระทรวงการท่องเที่ยวและกีฬา': 10,
 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200': 11}

In [37]:
train_data

array([['บริเวณนราธิวาส  แยกถนนจันทน์ ใกล้สวนสาธารณะช่องนนทรี ทางเท้าเน่ามาก',
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ['บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแถวหลายสายมากมาย ทำให้สองแถวชอบจอดรอผู้โดยสาร และผู้โดยสารที่ต้องเปลี่ยนสายสองแถวก็จะมารอตรงนี้เช่นกัน ทำให้รถติด แต่ก็จำเป็นสำหรับคนในย่านมาก ถ้ามีการพัฒนาพื้นที่เปลี่ยนถ่ายให้ดี ลดการรบกวนจราจรแต่ยังตอบสนองวิถีชีวิตเดิมได้จะดีมาก',
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ['เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซนต์หลุยส์',
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ...,
       ['หมาจรในซอยเฉลิมพระเกียรติร.9 42 นี้ดุ วิ่งไล่กัดคนเดินผ่าน เดือดร้อนคนแถวนั้น',
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ['แท็กซี่จอดกีดขวางป้ายรถเมล์',
        list([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ['มีขอทานบนสะพานลอยหน้าโลตัสปิ่นเกล้า',
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]],
      shape=(204622, 2), dtype=object)

# Model 3 WangchanBERTa

We ask you to train a WangchanBERTa-based model.

We recommend you use the thaixtransformers fork (which we used in the PoS homework).
https://github.com/PyThaiNLP/thaixtransformers

The structure of the code will be very similar to the PoS homework. You will also find the huggingface [tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) useful. Or you can also add a softmax layer by yourself just like in the previous homework.

Which WangchanBERTa model will you use? Why? (Don't forget to clean your text accordingly).

**Ans:**
`airesearch/wangchanberta-base-att-spm-uncased` because it is suitable for text classification. It was train on these datasets for Multiclass text classification:
- `wisesight_sentiment`: 4-class text classification task (positive, neutral, negative, and question) based on social media posts and tweets
- `wongnai_reivews`: Users' review rating classification task (scale is ranging from 1 to 5)
- `generated_reviews_enth`: Generated users' review rating classification task (scale is ranging from 1 to 5)

In [40]:
import time
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, DataCollatorWithPadding, AutoModel
from tqdm import tqdm
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from torchmetrics import Accuracy

from thaixtransformers import Tokenizer
from torch import nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'thaixtransformers'

In [58]:
# tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
tokenizer = Tokenizer("airesearch/wangchanberta-base-att-spm-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.


In [46]:
tokenizer(data[0,0])

{'input_ids': [5, 10, 2856, 3, 265, 3, 265, 3, 1992, 8, 330, 28, 2753, 12, 8, 10, 3, 743, 2508, 3441, 8, 10, 3, 1276, 714, 8321, 110, 8, 3310, 2148, 14999, 8, 10, 1836, 4813, 11, 570, 8, 10, 177, 8, 9484, 2301, 2390, 5824, 204, 11545, 8, 10, 313, 2801, 11, 4573, 8, 10, 177, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [47]:
for i, sample in tqdm(enumerate(data)):
    cleaned_sentence = sample[0].replace('ํา', "ำ") # only cleaning needed for WangchanBERTa
    data[i, 0] = tokenizer(cleaned_sentence) # no padding as we will use DataCollatorWithPadding to pad within batch

13385it [00:02, 5829.12it/s]


In [48]:
X = data[:, 0]
y = data[:, 1]

X.shape, y.shape

((13385,), (13385,))

In [49]:
X[0]

{'input_ids': [5, 10, 2856, 3, 265, 3, 265, 3, 1992, 8, 330, 28, 2753, 12, 8, 10, 3, 743, 2508, 3441, 8, 10, 3, 1276, 714, 8321, 110, 8, 3310, 2148, 14999, 8, 10, 1836, 4813, 11, 570, 8, 10, 177, 8, 9484, 2301, 2390, 5824, 204, 11545, 8, 10, 313, 2801, 11, 4573, 8, 10, 177, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [50]:
X_train: np.ndarray
y_train: np.ndarray
X_val: np.ndarray
y_val: np.ndarray
X_test: np.ndarray
y_test: np.ndarray

X_train, X_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(10708,) (10708,) (1338,) (1338,) (1339,) (1339,)


In [51]:
class CallCenterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val) for key, val in self.encodings[idx].items()
            if key in ['input_ids', 'attention_mask'] # take only input_ids and attention_mask fields
        }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [76]:
train_dataset = CallCenterDataset(X_train, y_train)
val_dataset = CallCenterDataset(X_val, y_val)
test_dataset = CallCenterDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator)

In [83]:
train_loader.dataset[0]['input_ids']

tensor([   5,   10, 4811,    8,   10, 3240,    8,   10,  177, 7268, 4090,  879,
           6])

In [84]:
train_loader.dataset[0]['labels']

tensor(1)

In [78]:
class BaseModel(LightningModule):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          learning_rate: float = 2e-5
    ):
        super().__init__()
        self.save_hyperparameters()

        self.encoder = AutoModel.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def get_embeddings(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids, attention_mask)
        hidden_states = outputs.last_hidden_state

        # [CLS] token is the first token in the sequence (index 0)
        cls_embeddings = hidden_states[:, 0, :]  # [batch_size, hidden_size]
        
        return cls_embeddings

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

    def forward(self, input_ids, attention_mask):
        return self.get_embeddings(input_ids, attention_mask)

In [ ]:
class LMWithLinearClassfier(BaseModel):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          ckpt_path: str = None,
          learning_rate: float = 2e-5,
          freeze_encoder_weights: bool = False
    ):
        super().__init__(
            model_name,
            learning_rate
        )
        self.save_hyperparameters()

        if ckpt_path:
            checkpoint = torch.load(ckpt_path)
            encoder_state_dict = {k.replace("encoder.", ""): v for k, v in checkpoint["state_dict"].items() if k.startswith("encoder.")}
            self.encoder.load_state_dict(encoder_state_dict)

        self.linear_layer = nn.Linear(768, 25)

        if freeze_encoder_weights:
          self.freeze_weights(self.encoder)  # Freeze model

        self.accuracy = Accuracy(task='multiclass', num_classes=25)

    def freeze_weights(self, model):
        for param in model.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        cls_embeddings = self.get_embeddings(input_ids, attention_mask)
        logits = self.linear_layer(cls_embeddings)
        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.cross_entropy(logits, labels)
        self.log('train_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('train_acc', acc, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.binary_cross_entropy_with_logits(logits, labels)
        self.log('val_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('val_acc', acc, prog_bar=True)

        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        logits = self.forward(input_ids, attention_mask)

        loss = F.binary_cross_entropy_with_logits(logits, labels)
        self.log('test_loss', loss, prog_bar=True)

        acc = self.accuracy(logits, labels)
        self.log('test_acc', acc, prog_bar=True)

        return loss

In [92]:
model = LMWithLinearClassfier(
    'airesearch/wangchanberta-base-att-spm-uncased',
    ckpt_path=None,
    freeze_encoder_weights=False
)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [93]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_acc",  # Metric to monitor
    mode="max",  # "min" for loss, "max" for accuracy
    save_top_k=1,  # Save only the best model(s)
    save_weights_only=True, # Saves only weights, not the entire model
    dirpath="./checkpoints/", # Path where the checkpoints will be saved
    filename="best_pretrained_w_linear_model-{epoch}-{val_acc:.2f}", # Customized name for the checkpoint
    verbose=True,
)

linear_trainer = Trainer(
    max_epochs=5,
    accelerator='auto',
    callbacks=[checkpoint_callback], # Add the ModelCheckpoint callback
    gradient_clip_val=1.0,
    precision=16, # Mixed precision training
    devices=1,
)

In [94]:
start_time = time.time()
# Train the model
linear_trainer.fit(model, train_loader, val_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Total time: 210.2021267414093


In [96]:
start_time = time.time()
result = linear_trainer.test(model, test_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6818521022796631     │
│         test_loss         │     1.19138503074646      │
└───────────────────────────┴───────────────────────────┘

Total time: 1.7742555141448975
